<a href="https://colab.research.google.com/github/Mekondjo-EDSA/DSI_Capstone_project/blob/main/capstone_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [1]:
import os
import csv
import json
import pickle
import shutil
import random
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime,timedelta
from sklearn.impute import SimpleImputer
from matplotlib.gridspec import GridSpec
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_extraction.text import CountVectorizer


In [9]:
train_ben= pd.read_csv('/content/drive/MyDrive/capstone_dataset/Train_Beneficiarydata-1542865627584.csv')
train_inpat= pd.read_csv('/content/drive/MyDrive/capstone_dataset/Train_Inpatientdata-1542865627584.csv')
train_outpat= pd.read_csv('/content/drive/MyDrive/capstone_dataset/Train_Outpatientdata-1542865627584.csv')
train_y= pd.read_csv('/content/drive/MyDrive/capstone_dataset/Train-1542865627584.csv')

In [10]:
print("Shape of Train Beneficiary file Data:",train_ben.shape)
print("Shape of Train In-patient file Data:", train_inpat.shape)
print("Shape of Train Out-patient file Data:", train_outpat.shape)
print("Shape of Train file Data:", train_y.shape)

Shape of Train Beneficiary file Data: (138556, 25)
Shape of Train In-patient file Data: (40474, 30)
Shape of Train Out-patient file Data: (517737, 27)
Shape of Train file Data: (5410, 2)


In [11]:
out_prov= np.unique(train_outpat['Provider'])
print("The number of Unique Providers in the Train_Outpat file:", len(out_prov))

in_prov= np.unique(train_inpat['Provider'])
print("The number of Unique Providers in the Train_Inpat file", len(in_prov))

com_prov= set(out_prov).intersection(set(in_prov))
print("The number of Providers common to both the Inpat and Outpat files:",len(com_prov))

uni_ele= len(out_prov)+len(in_prov)-len(com_prov)
print("Total Number of Unique Providers in Outpatient and Inpatient datasets Together:",uni_ele)

The number of Unique Providers in the Train_Outpat file: 5012
The number of Unique Providers in the Train_Inpat file 2092
The number of Providers common to both the Inpat and Outpat files: 1694
Total Number of Unique Providers in Outpatient and Inpatient datasets Together: 5410


In [15]:
#Check the columns present in the dataset
print("The columns in the Outpatient Dataset are:",train_outpat.columns)
print("="*100)
print("The columns in the Inpatient Dataset are:",train_inpat.columns)
print("="*100)
print("The columns in the Beneficiary Dataset are:",train_ben.columns)

The columns in the Outpatient Dataset are: Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode'],
      dtype='object')
The columns in the Inpatient Dataset are: Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
  

In [14]:
#Checking each of the columns in the Outaptient dataset if they are present in the Inpatient Dataset
c_o=[]
for o in train_outpat.columns:
    if o in train_inpat.columns:
        c_o.append(o)

#Checking each of the columns in the Inpatient dataset if they are present in the Outpatient dataset
c_i=[]
for i in train_inpat.columns:
    if i in train_outpat.columns:
        c_i.append(i)
print("Cols of Outpatient dataset also present in Inpatient dataset",len(c_o))
print("Cols of Inpatient dataset also present in Outpatient dataset",len(c_i))

#Checking for common column names in the outpatient and the inpatient datasets 
c_s= set(c_o).intersection(set(c_i))
c_s= list(c_s)
print("Common columns between the outpatient and the inpatient datasets",len(c_s))

Cols of Outpatient dataset also present in Inpatient dataset 27
Cols of Inpatient dataset also present in Outpatient dataset 27
Common columns between the outpatient and the inpatient datasets 27


In [17]:
#Merging the Outpatient and the Inpatient datasets based on the common columns between both the datasets. 
#We will be doing an outer merge as we need to take the union of all the elements in both the datasets 

train_fin_df= pd.merge(train_inpat,train_outpat,left_on=c_s,right_on=c_s,how='outer')
train_fin_df.shape


(558211, 30)

In [18]:
train_fin_df.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [19]:
train_fin= pd.merge(train_fin_df,train_ben, left_on='BeneID',right_on= 'BeneID',how='outer')
train_fin.shape


(558211, 54)

In [20]:
#Merging the Y variable with the final dataset
train_fin= pd.merge(train_fin,train_y,left_on= 'Provider',right_on='Provider',how='outer')
train_fin.shape

(558211, 55)

In [21]:
#Storing the final dataframe as a pickled file
with open('/content/drive/MyDrive/Colab Notebooks/train_fin.pkl','wb') as tr_df:
    pickle.dump(train_fin,tr_df)


In [22]:
#Loading the pickled file
with open('/content/drive/MyDrive/Colab Notebooks/train_fin.pkl','rb') as tr_df:
    train_fin= pickle.load(tr_df)

In [23]:
train_fin.shape

(558211, 55)

In [24]:
#Splitting the Data into Train and Cross Validate Datasets
y = train_fin['PotentialFraud']
train_fin.drop(['PotentialFraud'],axis=1, inplace= True)

In [25]:
train_fin,cv_fin,train_y,cv_y= train_test_split(train_fin,y,test_size=0.2,stratify=y,random_state=42)
print(train_fin.shape)
print(train_y.shape)
print(cv_fin.shape)
print(cv_y.shape)

(446568, 54)
(446568,)
(111643, 54)
(111643,)


In [26]:
train_fin.reset_index(drop=True,inplace=True)
train_y.reset_index(drop=True,inplace=True)

cv_fin.reset_index(drop=True,inplace=True)
cv_y.reset_index(drop=True,inplace=True)

In [27]:

with open('/content/drive/MyDrive/Colab Notebooks/train_x.pkl','wb') as tr_df:
    pickle.dump(train_fin,tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_x.pkl','wb') as cv_df:
    pickle.dump(cv_fin,cv_df)
with open('/content/drive/MyDrive/Colab Notebooks/train_y.pkl','wb') as tr_y:
    pickle.dump(train_y,tr_y)
with open('/content/drive/MyDrive/Colab Notebooks/cv_y.pkl','wb') as c_y:
    pickle.dump(cv_y,c_y)


In [28]:

with open('/content/drive/MyDrive/Colab Notebooks/train_x.pkl','rb') as tr_df:
    train_fin= pickle.load(tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_x.pkl','rb') as cv_df:
    cv_fin= pickle.load(cv_df)
with open('/content/drive/MyDrive/Colab Notebooks/train_y.pkl','rb') as tr_y:
    train_y= pickle.load(tr_y)
with open('/content/drive/MyDrive/Colab Notebooks/cv_y.pkl','rb') as c_y:
    cv_y= pickle.load(c_y)


In [29]:
#Checking for percentage of NAN values in each column in the train dataset
na_perc= np.round(((train_fin.isna().sum())/train_fin.shape[0])*100,2)
na_perc_df= na_perc.to_frame()
na_perc_df.reset_index(inplace= True)
na_perc_df.columns= ["col_name","NAN_percentage"]
print(na_perc_df)

                           col_name  NAN_percentage
0                            BeneID            0.00
1                           ClaimID            0.00
2                      ClaimStartDt            0.00
3                        ClaimEndDt            0.00
4                          Provider            0.00
5            InscClaimAmtReimbursed            0.00
6                AttendingPhysician            0.27
7                OperatingPhysician           79.50
8                    OtherPhysician           64.24
9                       AdmissionDt           92.72
10            ClmAdmitDiagnosisCode           73.88
11                DeductibleAmtPaid            0.16
12                      DischargeDt           92.72
13               DiagnosisGroupCode           92.72
14               ClmDiagnosisCode_1            1.86
15               ClmDiagnosisCode_2           35.04
16               ClmDiagnosisCode_3           56.47
17               ClmDiagnosisCode_4           70.52
18          

In [30]:
#find the column numbers where the NAN_percentage is Zero
na_col=[]
na_perc= np.round(((train_fin.isna().sum())/train_fin.shape[0])*100,2)
na_perc_df= na_perc.to_frame()
na_perc_df.reset_index(inplace= True)
na_perc_df.columns= ["col_name","na_percentage"]
for i in range(na_perc_df.shape[0]):
    if na_perc_df.iloc[i,1] == 0:
        na_col.append(i)  

print(na_col)

[0, 1, 2, 3, 4, 5, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]


In [31]:
#Del the columns with 0% NAN values from the newly created na_perc_df dataframe
na_perc_df.drop(index=na_col,inplace=True)
na_perc_df.reset_index(drop=True,inplace=True)
print(na_perc_df)

                 col_name  na_percentage
0      AttendingPhysician           0.27
1      OperatingPhysician          79.50
2          OtherPhysician          64.24
3             AdmissionDt          92.72
4   ClmAdmitDiagnosisCode          73.88
5       DeductibleAmtPaid           0.16
6             DischargeDt          92.72
7      DiagnosisGroupCode          92.72
8      ClmDiagnosisCode_1           1.86
9      ClmDiagnosisCode_2          35.04
10     ClmDiagnosisCode_3          56.47
11     ClmDiagnosisCode_4          70.52
12     ClmDiagnosisCode_5          79.93
13     ClmDiagnosisCode_6          84.86
14     ClmDiagnosisCode_7          88.13
15     ClmDiagnosisCode_8          90.42
16     ClmDiagnosisCode_9          92.50
17    ClmDiagnosisCode_10          99.09
18     ClmProcedureCode_1          95.80
19     ClmProcedureCode_2          99.02
20     ClmProcedureCode_3          99.83
21     ClmProcedureCode_4          99.98
22     ClmProcedureCode_5         100.00
23     ClmProced

In [32]:
#Checking for the percentage of nan values in each of the columns in the CV Data
na_perc_cv= np.round(((cv_fin.isna().sum())/cv_fin.shape[0])*100,2)
na_perc_df_cv= na_perc_cv.to_frame()
na_perc_df_cv.reset_index(inplace= True)
na_perc_df_cv.columns= ["col_name","na_percentage"]
print(na_perc_df_cv)

                           col_name  na_percentage
0                            BeneID           0.00
1                           ClaimID           0.00
2                      ClaimStartDt           0.00
3                        ClaimEndDt           0.00
4                          Provider           0.00
5            InscClaimAmtReimbursed           0.00
6                AttendingPhysician           0.27
7                OperatingPhysician          79.47
8                    OtherPhysician          64.13
9                       AdmissionDt          92.88
10            ClmAdmitDiagnosisCode          73.80
11                DeductibleAmtPaid           0.16
12                      DischargeDt          92.88
13               DiagnosisGroupCode          92.88
14               ClmDiagnosisCode_1           1.92
15               ClmDiagnosisCode_2          35.04
16               ClmDiagnosisCode_3          56.43
17               ClmDiagnosisCode_4          70.53
18               ClmDiagnosisCo

In [33]:
na_col_cv=[]
na_perc_cv= np.round(((cv_fin.isna().sum())/cv_fin.shape[0])*100,2)
na_perc_df_cv= na_perc_cv.to_frame()
na_perc_df_cv.reset_index(inplace= True)
na_perc_df_cv.columns= ["col_name","na_percentage"]
for i in range(na_perc_df_cv.shape[0]):
    if na_perc_df_cv.iloc[i,1] == 0:
        na_col_cv.append(i)  

print(na_col_cv)

[0, 1, 2, 3, 4, 5, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]


In [34]:
na_perc_df_cv.drop(index=na_col_cv,inplace=True)
na_perc_df_cv.reset_index(drop=True,inplace=True)
print(na_perc_df_cv)

                 col_name  na_percentage
0      AttendingPhysician           0.27
1      OperatingPhysician          79.47
2          OtherPhysician          64.13
3             AdmissionDt          92.88
4   ClmAdmitDiagnosisCode          73.80
5       DeductibleAmtPaid           0.16
6             DischargeDt          92.88
7      DiagnosisGroupCode          92.88
8      ClmDiagnosisCode_1           1.92
9      ClmDiagnosisCode_2          35.04
10     ClmDiagnosisCode_3          56.43
11     ClmDiagnosisCode_4          70.53
12     ClmDiagnosisCode_5          80.02
13     ClmDiagnosisCode_6          84.95
14     ClmDiagnosisCode_7          88.21
15     ClmDiagnosisCode_8          90.45
16     ClmDiagnosisCode_9          92.56
17    ClmDiagnosisCode_10          99.14
18     ClmProcedureCode_1          95.91
19     ClmProcedureCode_2          99.02
20     ClmProcedureCode_3          99.83
21     ClmProcedureCode_4          99.98
22     ClmProcedureCode_5         100.00
23     ClmProced

In [35]:
clm_proc= ['ClmProcedureCode_1','ClmProcedureCode_2','ClmProcedureCode_3','ClmProcedureCode_4','ClmProcedureCode_5','ClmProcedureCode_6']

In [36]:
#Isolating all the claim procedure columnns of the train and crossvalidate datasets into seperate dataframes
tr_clm_pr = train_fin[clm_proc]
cv_clm_pr = cv_fin[clm_proc]
print(tr_clm_pr.shape)
print(cv_clm_pr.shape)

(446568, 6)
(111643, 6)


In [37]:
#Creating a new column called '#_Procedures' to save the counts for each row where the counts of the non-nan values in each of the clm_proc are stored
tr_clm_pr['#_Procedures']= np.zeros(len(train_fin['ClmProcedureCode_1']))

for i in tqdm(range(len(tr_clm_pr['ClmProcedureCode_1']))):
    count= 0
    for j in range(len(clm_proc)):
        if pd.isnull(tr_clm_pr.iloc[i,j])== False:
            count=count+1
    
    tr_clm_pr['#_Procedures'][i]= count


100%|██████████| 446568/446568 [03:37<00:00, 2049.27it/s]


In [38]:
cv_clm_pr['#_Procedures']= np.zeros(len(cv_fin['ClmProcedureCode_1']))

#Looping through each of the claim procedure columns and each of the observations
#Counting the number of non-na values in each of the clm_proc columns in the each of the obs
#Storing the count values in a seperate column titled '#_Procedures'
for i in tqdm(range(len(cv_clm_pr['ClmProcedureCode_1']))):
    count= 0
    for j in range(len(clm_proc)):
        if pd.isnull(cv_clm_pr.iloc[i,j])== False:
            count=count+1
    
    cv_clm_pr['#_Procedures'][i]= count

100%|██████████| 111643/111643 [00:55<00:00, 2014.09it/s]


In [39]:
train_fin['#_Procedures']= tr_clm_pr['#_Procedures']
cv_fin['#_Procedures']= cv_clm_pr['#_Procedures']

In [40]:
print(np.unique(train_fin['#_Procedures']))
print(np.unique(cv_fin['#_Procedures']))

[0. 1. 2. 3. 4. 5.]
[0. 1. 2. 3. 4. 5.]


In [41]:
#drop claim procedure code variables
train_fin.drop(clm_proc, axis=1, inplace= True)
cv_fin.drop(clm_proc, axis=1, inplace= True)

In [43]:
clm_diag= ['ClmDiagnosisCode_1','ClmDiagnosisCode_2','ClmDiagnosisCode_3','ClmDiagnosisCode_4','ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7','ClmDiagnosisCode_8','ClmDiagnosisCode_9','ClmDiagnosisCode_10']


In [44]:
tr_clm_dg= train_fin[clm_diag]
cv_clm_dg= cv_fin[clm_diag]
print(tr_clm_dg.shape)
print(cv_clm_dg.shape)

(446568, 10)
(111643, 10)


In [45]:
tr_clm_dg['#_DiagnosisCodes']= np.zeros(len(tr_clm_dg['ClmDiagnosisCode_1']))

#Looping through each of the claim diagnosis columns and each of the observations
#Counting the number of non-na values in each of the clm_diag columns in the each of the obs
#Storing the count values in a seperate column titled '#_DiagnosisCodes'
for i in tqdm(range(len(tr_clm_dg['ClmDiagnosisCode_1']))):
    count= 0
    for j in range(len(clm_diag)):
        if pd.isnull(tr_clm_dg.iloc[i,j])== False:
            count=count+1
    
    tr_clm_dg['#_DiagnosisCodes'][i]= count

100%|██████████| 446568/446568 [04:36<00:00, 1612.20it/s]


In [46]:


cv_clm_dg['#_DiagnosisCodes']= np.zeros(len(cv_clm_dg['ClmDiagnosisCode_1']))

#Looping through each of the claim diagnosis columns and each of the observations
#Counting the number of non-na values in each of the clm_diag columns in the each of the obs
#Storing the count values in a seperate column titled '#_DiagnosisCodes'
for i in tqdm(range(len(cv_clm_dg['ClmDiagnosisCode_1']))):
    count= 0
    for j in range(len(clm_diag)):
        if pd.isnull(cv_clm_dg.iloc[i,j])== False:
            count=count+1
    
    cv_clm_dg['#_DiagnosisCodes'][i]= count



100%|██████████| 111643/111643 [01:14<00:00, 1490.68it/s]


In [47]:
train_fin['#_DiagnosisCodes']= tr_clm_dg['#_DiagnosisCodes']
cv_fin['#_DiagnosisCodes']= cv_clm_dg['#_DiagnosisCodes']

In [48]:
#Dropping the 10 Claim Diagnosis codes variables
train_fin.drop(clm_diag,axis=1,inplace= True)
cv_fin.drop(clm_diag,axis=1,inplace= True)

In [49]:
train_fin["HospitalDays"]= pd.to_datetime(train_fin['DischargeDt']) - pd.to_datetime(train_fin['AdmissionDt'])
train_fin["HospitalDays"]= train_fin["HospitalDays"].dt.days

In [50]:
cv_fin["HospitalDays"]= pd.to_datetime(cv_fin['DischargeDt']) - pd.to_datetime(cv_fin['AdmissionDt'])
cv_fin["HospitalDays"]= cv_fin["HospitalDays"].dt.days

In [51]:
train_fin.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,#_Procedures,#_DiagnosisCodes,HospitalDays
0,BENE22189,CLM164897,2009-01-29,2009-01-29,PRV54273,90,PHY419362,NaN,PHY384386,NaN,...,1,2,2,5030,1068,1090,560,0.0,1.0,NaN
1,BENE156743,CLM79687,2009-12-15,2009-12-21,PRV56438,5000,PHY410043,NaN,NaN,2009-12-15,...,2,2,2,5000,1068,70,0,0.0,9.0,6.0
2,BENE157334,CLM724410,2009-12-11,2009-12-31,PRV53918,400,PHY351111,NaN,NaN,NaN,...,2,1,1,0,0,3870,540,0.0,3.0,NaN
3,BENE30606,CLM580231,2009-09-15,2009-09-15,PRV55004,30,PHY342769,NaN,PHY371896,NaN,...,1,2,2,3000,1068,1680,140,0.0,5.0,NaN
4,BENE11648,CLM76557,2009-11-19,2009-11-22,PRV51149,2000,PHY332294,PHY332294,NaN,2009-11-19,...,2,2,2,2000,1068,1160,650,1.0,9.0,3.0


In [52]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n1.pkl','wb') as tr_df:
    pickle.dump(train_fin,tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n1.pkl','wb') as cv_df:
    pickle.dump(cv_fin,cv_df)

In [53]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n1.pkl','rb') as tr_df:
    train_fin1= pickle.load(tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n1.pkl','rb') as cv_df:
    cv_fin1= pickle.load(cv_df)

In [54]:
print(train_fin1['HospitalDays'].isna().sum())
print(cv_fin1['HospitalDays'].isna().sum())

414039
103698


In [55]:
#Imputing the NA values in the HospitalDates variable with 0 values
train_fin1["HospitalDays"]=train_fin1["HospitalDays"].fillna(0)
cv_fin1["HospitalDays"]=cv_fin1["HospitalDays"].fillna(0)

In [56]:
#Categorizing the variable as per weeks as it introduces 
#ordinality in the feature as well as seperates out the inpatient and the outpatient data 
for i in tqdm(range(len(train_fin1['HospitalDays']))):
    if train_fin1['HospitalDays'][i]==0.0:
        train_fin1["HospitalDays"][i]= 0
    if train_fin1["HospitalDays"][i]>0 and train_fin1["HospitalDays"][i]<=7:
        train_fin1["HospitalDays"][i]= 1
    elif train_fin1["HospitalDays"][i]>7 and train_fin1["HospitalDays"][i]<=14:
        train_fin1["HospitalDays"][i]= 2 
    elif train_fin1["HospitalDays"][i]>14 and train_fin1["HospitalDays"][i]<=21:
        train_fin1["HospitalDays"][i]= 3
    elif train_fin1["HospitalDays"][i]>21 and train_fin1["HospitalDays"][i]<=28:
        train_fin1["HospitalDays"][i]= 4
    elif train_fin1["HospitalDays"][i]>28:
        train_fin1["HospitalDays"][i]= 5

100%|██████████| 446568/446568 [02:11<00:00, 3387.89it/s]


In [57]:
train_fin1= train_fin1.rename(columns={'HospitalDays':'HospitalWeeks'})
cv_fin1= cv_fin1.rename(columns={'HospitalDays':'HospitalWeeks'})

In [58]:
#Dropping the Admission Date and the Discharge Date columns from the dataset
train_fin1.drop(['AdmissionDt','DischargeDt'], axis=1, inplace=True)
cv_fin1.drop(['AdmissionDt','DischargeDt'], axis=1, inplace=True)

In [59]:
#Imputing the Attending Physician with Mode or MostFrequent strategy using SimpleImputer
cat_imp= SimpleImputer(missing_values= np.nan, strategy= 'most_frequent')
train_fin1["AttendingPhysician"]= cat_imp.fit_transform(train_fin1['AttendingPhysician'].values.reshape(-1,1))[:,0]

In [60]:
train_fin1["AttendingPhysician"].isna().sum()

0

In [63]:
for i in tqdm(range(len(train_fin1["AttendingPhysician"]))):
    if pd.isnull(train_fin1["OperatingPhysician"][i])==True and pd.isnull(train_fin1["OtherPhysician"][i])==True:
        train_fin1["IllnessNature"][i]= "simple"
    elif pd.isnull(train_fin1["OperatingPhysician"][i])==True and pd.isnull(train_fin1["OtherPhysician"][i])==False:
        train_fin1["IllnessNature"][i]= "operating"
    elif pd.isnull(train_fin1["OperatingPhysician"][i])==False and pd.isnull(train_fin1["OtherPhysician"][i])==True:
        train_fin1["IllnessNature"][i]= "comorbid"
    elif pd.isnull(train_fin1["OperatingPhysician"][i])==False and pd.isnull(train_fin1["OtherPhysician"][i])==False:
        train_fin1["IllnessNature"][i]= "operating&comorbid"

100%|██████████| 446568/446568 [00:12<00:00, 35662.81it/s]


In [64]:
for i in tqdm(range(len(cv_fin1["AttendingPhysician"]))):
    if pd.isnull(cv_fin1["OperatingPhysician"][i])==True and pd.isnull(cv_fin1["OtherPhysician"][i])==True:
        cv_fin1["IllnessNature"][i]= "simple"
    elif pd.isnull(cv_fin1["OperatingPhysician"][i])==True and pd.isnull(cv_fin1["OtherPhysician"][i])==False:
        cv_fin1["IllnessNature"][i]= "operating"
    elif pd.isnull(cv_fin1["OperatingPhysician"][i])==False and pd.isnull(cv_fin1["OtherPhysician"][i])==True:
        cv_fin1["IllnessNature"][i]= "comorbid"
    elif pd.isnull(cv_fin1["OperatingPhysician"][i])==False and pd.isnull(cv_fin1["OtherPhysician"][i])==False:
        cv_fin1["IllnessNature"][i]= "operating&comorbid"

100%|██████████| 111643/111643 [00:03<00:00, 35728.71it/s]


In [65]:
print(np.unique(train_fin1["IllnessNature"]))
print(np.unique(cv_fin1["IllnessNature"]))

['comorbid' 'operating' 'operating&comorbid' 'simple']
['comorbid' 'operating' 'operating&comorbid' 'simple']


In [66]:
train_fin1.drop(['OperatingPhysician','OtherPhysician'], axis=1,inplace=True)
cv_fin1.drop(['OperatingPhysician','OtherPhysician'], axis=1,inplace=True)

In [68]:
train_fin1.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,...,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,#_Procedures,#_DiagnosisCodes,HospitalWeeks,IllnessNature
0,BENE22189,CLM164897,2009-01-29,2009-01-29,PRV54273,90,PHY419362,25000,0.0,NaN,...,2,2,5030,1068,1090,560,0.0,1.0,0.0,operating
1,BENE156743,CLM79687,2009-12-15,2009-12-21,PRV56438,5000,PHY410043,486,1068.0,628,...,2,2,5000,1068,70,0,0.0,9.0,1.0,simple
2,BENE157334,CLM724410,2009-12-11,2009-12-31,PRV53918,400,PHY351111,NaN,0.0,NaN,...,1,1,0,0,3870,540,0.0,3.0,0.0,simple
3,BENE30606,CLM580231,2009-09-15,2009-09-15,PRV55004,30,PHY342769,NaN,0.0,NaN,...,2,2,3000,1068,1680,140,0.0,5.0,0.0,operating
4,BENE11648,CLM76557,2009-11-19,2009-11-22,PRV51149,2000,PHY332294,42823,1068.0,236,...,2,2,2000,1068,1160,650,1.0,9.0,1.0,comorbid


In [69]:
#Extracting the number of days from the claim start date and the claim end dates by taking a difference
#Converting the number of days during which the claim was in process into number of weeks

train_fin1["ClaimDays"]= (pd.to_datetime(train_fin1['ClaimEndDt']) - pd.to_datetime(train_fin1['ClaimStartDt'])).dt.days
cv_fin1["ClaimDays"]= (pd.to_datetime(cv_fin1['ClaimEndDt']) - pd.to_datetime(cv_fin1['ClaimStartDt'])).dt.days

In [70]:
train_fin1.drop(['ClaimEndDt','ClaimStartDt'],axis=1, inplace=True)
cv_fin1.drop(['ClaimEndDt','ClaimStartDt'],axis=1, inplace=True)

In [71]:
for i in tqdm(range(len(train_fin1['ClaimDays']))):
    if train_fin1['ClaimDays'][i]==0:
        train_fin1['ClaimDays'][i]=0 
    elif train_fin1['ClaimDays'][i]>0 and train_fin1['ClaimDays'][i]<=7:
        train_fin1['ClaimDays'][i]= 1
    elif train_fin1['ClaimDays'][i]>7 and train_fin1['ClaimDays'][i]<=14:
        train_fin1['ClaimDays'][i]= 2
    elif train_fin1['ClaimDays'][i]> 14 and train_fin1['ClaimDays'][i]<=21:
        train_fin1['ClaimDays'][i]= 3
    elif train_fin1['ClaimDays'][i]>21 and train_fin1['ClaimDays'][i]<=28:
        train_fin1['ClaimDays'][i]= 4
    elif train_fin1['ClaimDays'][i]>28 and train_fin1['ClaimDays'][i]<=35:
        train_fin1['ClaimDays'][i]=5
    elif train_fin1['ClaimDays'][i]>35:
        train_fin1['ClaimDays'][i]=6


100%|██████████| 446568/446568 [01:57<00:00, 3814.56it/s]


In [72]:
for i in tqdm(range(len(cv_fin1['ClaimDays']))):
    if cv_fin1['ClaimDays'][i]==0:
        cv_fin1['ClaimDays'][i]=0 
    elif cv_fin1['ClaimDays'][i]>0 and cv_fin1['ClaimDays'][i]<=7:
        cv_fin1['ClaimDays'][i]= 1
    elif cv_fin1['ClaimDays'][i]>7 and cv_fin1['ClaimDays'][i]<=14:
        cv_fin1['ClaimDays'][i]= 2
    elif cv_fin1['ClaimDays'][i]> 14 and cv_fin1['ClaimDays'][i]<=21:
        cv_fin1['ClaimDays'][i]= 3
    elif cv_fin1['ClaimDays'][i]>21 and cv_fin1['ClaimDays'][i]<=28:
        cv_fin1['ClaimDays'][i]= 4
    elif cv_fin1['ClaimDays'][i]>28 and cv_fin1['ClaimDays'][i]<=35:
        cv_fin1['ClaimDays'][i]=5
    elif cv_fin1['ClaimDays'][i]>35:
        cv_fin1['ClaimDays'][i]=6

100%|██████████| 111643/111643 [00:29<00:00, 3780.53it/s]


In [73]:
train_fin1= train_fin1.rename(columns={'ClaimDays':'ClaimWeeks'})
cv_fin1= cv_fin1.rename(columns={'ClaimDays':'ClaimWeeks'})

In [74]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n2.pkl','wb') as tr_df:
    pickle.dump(train_fin1,tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n2.pkl','wb') as cv_df:
    pickle.dump(cv_fin1,cv_df)

In [75]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n2.pkl','rb') as tr_df:
    train_fin2= pickle.load(tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n2.pkl','rb') as cv_df:
    cv_fin2= pickle.load(cv_df)

In [76]:
train_fin2['DiagnosisGroupCode']= train_fin2['DiagnosisGroupCode'].fillna(0)
cv_fin2['DiagnosisGroupCode']= cv_fin2['DiagnosisGroupCode'].fillna(0)

In [77]:
train_fin2['DiagnosisGroupCode']= train_fin2['DiagnosisGroupCode'].astype('str')
train_fin2['DiagnosisGroupCode']= train_fin2['DiagnosisGroupCode'].astype('str')

train_fin2['DiagnosisGroupCode'].describe()

count     446568
unique       735
top            0
freq      414039
Name: DiagnosisGroupCode, dtype: object

In [78]:
train_fin2['PotentialFraud']= train_y
cv_fin2['PotentialFraud']= cv_y

In [79]:
#Calculating the proabilities of each of the categories in a column
def response_coding(tr_data,cv_data,col,y):
    tr_yes_list=[]
    tr_no_list=[]
    cv_yes_list=[]
    cv_no_list=[]
    
    val_dict= dict(tr_data.groupby([col])[y].value_counts())
    
    for i in range(len(tr_data[col])):
        t_y= val_dict.get((tr_data[col][i],'Yes'),0.1)
        t_n= val_dict.get((tr_data[col][i],'No'),0.1)
        tr_yes_list.append(t_y/(t_y+t_n))
        tr_no_list.append(t_n/(t_y+t_n))

    for j in range(len(cv_data[col])):
        c_y= val_dict.get((cv_data[col][j],'Yes'),0.1)
        c_n= val_dict.get((cv_data[col][j],'No'),0.1)
        cv_yes_list.append(c_y/(c_y+c_n))
        cv_no_list.append(c_n/(c_y+c_n))  
    
    return tr_yes_list,tr_no_list,cv_yes_list,cv_no_list,val_dict

In [80]:
train_fin2['State_Yes']= np.zeros(len(train_fin2['State']))
train_fin2['State_No']= np.zeros(len(train_fin2['State']))

cv_fin2['State_Yes']= np.zeros(len(cv_fin2['State']))
cv_fin2['State_No']= np.zeros(len(cv_fin2['State']))

train_fin2['State_Yes'],train_fin2['State_No'],cv_fin2['State_Yes'],cv_fin2['State_No'],state_dict= response_coding(train_fin2,cv_fin2,'State','PotentialFraud')
print(state_dict)

{(1, 'No'): 5427, (1, 'Yes'): 2822, (2, 'No'): 430, (2, 'Yes'): 161, (3, 'No'): 5845, (3, 'Yes'): 1619, (4, 'No'): 4482, (4, 'Yes'): 1562, (5, 'Yes'): 24244, (5, 'No'): 16819, (6, 'Yes'): 2922, (6, 'No'): 2862, (7, 'No'): 3340, (7, 'Yes'): 1186, (8, 'No'): 728, (8, 'Yes'): 492, (9, 'No'): 364, (9, 'Yes'): 145, (10, 'No'): 17273, (10, 'Yes'): 14020, (11, 'No'): 10273, (11, 'Yes'): 3299, (12, 'No'): 1093, (12, 'Yes'): 240, (13, 'No'): 1514, (13, 'Yes'): 680, (14, 'No'): 12697, (14, 'Yes'): 6797, (15, 'No'): 7656, (15, 'Yes'): 3729, (16, 'No'): 3753, (16, 'Yes'): 2183, (17, 'No'): 3120, (17, 'Yes'): 805, (18, 'No'): 6924, (18, 'Yes'): 1356, (19, 'No'): 5000, (19, 'Yes'): 1375, (20, 'No'): 2181, (20, 'Yes'): 881, (21, 'No'): 6945, (21, 'Yes'): 2056, (22, 'Yes'): 6220, (22, 'No'): 4690, (23, 'No'): 10253, (23, 'Yes'): 6845, (24, 'No'): 3438, (24, 'Yes'): 2732, (25, 'No'): 4682, (25, 'Yes'): 684, (26, 'No'): 6150, (26, 'Yes'): 4236, (27, 'No'): 1128, (27, 'Yes'): 456, (28, 'No'): 2913, (28, 

In [81]:
train_fin2['County_Yes']= np.zeros(len(train_fin2['County']))
train_fin2['County_No']= np.zeros(len(train_fin2['County']))

cv_fin2['County_Yes']= np.zeros(len(cv_fin2['County']))
cv_fin2['County_No']= np.zeros(len(cv_fin2['County']))

train_fin2['County_Yes'],train_fin2['County_No'],cv_fin2['County_Yes'],cv_fin2['County_No'],county_dict= response_coding(train_fin2,cv_fin2,'County','PotentialFraud')
print(county_dict)

{(0, 'No'): 5333, (0, 'Yes'): 3885, (1, 'No'): 7, (1, 'Yes'): 4, (10, 'No'): 8580, (10, 'Yes'): 2561, (11, 'No'): 156, (11, 'Yes'): 60, (14, 'Yes'): 5, (20, 'No'): 6752, (20, 'Yes'): 3344, (25, 'Yes'): 31, (25, 'No'): 11, (30, 'No'): 3584, (30, 'Yes'): 1401, (34, 'No'): 8, (34, 'Yes'): 2, (40, 'No'): 4264, (40, 'Yes'): 1784, (50, 'No'): 4351, (50, 'Yes'): 1982, (55, 'No'): 17, (55, 'Yes'): 8, (60, 'No'): 7090, (60, 'Yes'): 2535, (70, 'No'): 4353, (70, 'Yes'): 1066, (80, 'No'): 3473, (80, 'Yes'): 1234, (84, 'Yes'): 7, (84, 'No'): 1, (88, 'Yes'): 52, (88, 'No'): 16, (90, 'No'): 5635, (90, 'Yes'): 3250, (100, 'No'): 3985, (100, 'Yes'): 1403, (110, 'No'): 1764, (110, 'Yes'): 1267, (111, 'Yes'): 61, (111, 'No'): 41, (113, 'No'): 9, (113, 'Yes'): 4, (117, 'No'): 9, (117, 'Yes'): 3, (120, 'No'): 3970, (120, 'Yes'): 1844, (130, 'Yes'): 3655, (130, 'No'): 2979, (131, 'No'): 11, (131, 'Yes'): 8, (140, 'No'): 2789, (140, 'Yes'): 1495, (141, 'No'): 4628, (141, 'Yes'): 2550, (150, 'No'): 4653, (150

In [82]:
train_fin2['DGC_Yes']= np.zeros(len(train_fin2['DiagnosisGroupCode']))
train_fin2['DGC_No']= np.zeros(len(train_fin2['DiagnosisGroupCode']))

cv_fin2['DGC_Yes']= np.zeros(len(cv_fin2['DiagnosisGroupCode']))
cv_fin2['DGC_No']= np.zeros(len(cv_fin2['DiagnosisGroupCode']))

train_fin2['DGC_Yes'],train_fin2['DGC_No'],cv_fin2['DGC_Yes'],cv_fin2['DGC_No'],dgc_dict= response_coding(train_fin2,cv_fin2,'DiagnosisGroupCode','PotentialFraud')
print(dgc_dict)

{('0', 'No'): 262599, ('0', 'Yes'): 151440, ('000', 'Yes'): 65, ('000', 'No'): 43, ('001', 'Yes'): 6, ('001', 'No'): 2, ('002', 'No'): 9, ('002', 'Yes'): 6, ('003', 'Yes'): 8, ('003', 'No'): 5, ('004', 'Yes'): 9, ('004', 'No'): 5, ('005', 'Yes'): 6, ('005', 'No'): 5, ('006', 'No'): 5, ('006', 'Yes'): 3, ('007', 'Yes'): 5, ('007', 'No'): 2, ('008', 'Yes'): 9, ('008', 'No'): 3, ('009', 'No'): 4, ('009', 'Yes'): 4, ('010', 'No'): 5, ('010', 'Yes'): 2, ('011', 'No'): 8, ('011', 'Yes'): 5, ('012', 'Yes'): 10, ('012', 'No'): 3, ('013', 'Yes'): 7, ('013', 'No'): 1, ('020', 'Yes'): 18, ('020', 'No'): 7, ('021', 'Yes'): 20, ('021', 'No'): 7, ('022', 'Yes'): 22, ('022', 'No'): 10, ('023', 'Yes'): 10, ('023', 'No'): 8, ('024', 'Yes'): 18, ('024', 'No'): 5, ('025', 'No'): 18, ('025', 'Yes'): 16, ('026', 'Yes'): 19, ('026', 'No'): 10, ('027', 'No'): 18, ('027', 'Yes'): 12, ('028', 'Yes'): 23, ('028', 'No'): 14, ('029', 'Yes'): 21, ('029', 'No'): 18, ('030', 'Yes'): 17, ('030', 'No'): 15, ('031', 'Y

In [83]:
train_fin2['BID_Yes']= np.zeros(len(train_fin2['BeneID']))
train_fin2['BID_No']= np.zeros(len(train_fin2['BeneID']))

train_fin2['BID_Yes']= np.zeros(len(train_fin2['BeneID']))
train_fin2['BID_No']= np.zeros(len(train_fin2['BeneID']))

train_fin2['BID_Yes'],train_fin2['BID_No'],cv_fin2['BID_Yes'],cv_fin2['BID_No'],bid_dict= response_coding(train_fin2,cv_fin2,'BeneID','PotentialFraud')


In [84]:
train_fin2['CID_Yes']= np.zeros(len(train_fin2['ClaimID']))
train_fin2['CID_No']= np.zeros(len(train_fin2['ClaimID']))

cv_fin2['CID_Yes']= np.zeros(len(cv_fin2['ClaimID']))
cv_fin2['CID_No']= np.zeros(len(cv_fin2['ClaimID']))

train_fin2['CID_Yes'],train_fin2['CID_No'],cv_fin2['CID_Yes'],cv_fin2['CID_No'],cid_dict= response_coding(train_fin2,cv_fin2,'ClaimID','PotentialFraud')

In [85]:
train_fin2['Pvr_Yes']= np.zeros(len(train_fin2['Provider']))
train_fin2['Pvr_No']= np.zeros(len(train_fin2['Provider']))

cv_fin2['Pvr_Yes']= np.zeros(len(cv_fin2['Provider']))
cv_fin2['Pvr_No']= np.zeros(len(cv_fin2['Provider']))

train_fin2['Pvr_Yes'],train_fin2['Pvr_No'],cv_fin2['Pvr_Yes'],cv_fin2['Pvr_No'],pvr_dict= response_coding(train_fin2,cv_fin2,'Provider','PotentialFraud')

In [86]:
train_fin2['Ap_Yes']= np.zeros(len(train_fin2['AttendingPhysician']))
train_fin2['Ap_No']= np.zeros(len(train_fin2['AttendingPhysician']))

cv_fin2['Ap_Yes']= np.zeros(len(cv_fin2['AttendingPhysician']))
cv_fin2['Ap_No']= np.zeros(len(cv_fin2['AttendingPhysician']))


train_fin2['Ap_Yes'],train_fin2['Ap_No'],cv_fin2['Ap_Yes'],cv_fin2['Ap_No'],ap_dict= response_coding(train_fin2,cv_fin2,'AttendingPhysician','PotentialFraud')

In [87]:
train_fin2.drop(['State','County','DiagnosisGroupCode','BeneID','ClaimID','Provider','AttendingPhysician'], axis=1, inplace=True)
cv_fin2.drop(['State','County','DiagnosisGroupCode','BeneID','ClaimID','Provider','AttendingPhysician'], axis=1, inplace=True)

In [88]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n3.pkl','wb') as tr_df:
    pickle.dump(train_fin2,tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n3.pkl','wb') as cv_df:
    pickle.dump(cv_fin2,cv_df)

In [89]:
#Dropping the NoOfMonths
train_fin2.drop(['NoOfMonths_PartACov','NoOfMonths_PartBCov'],axis=1, inplace= True)
cv_fin2.drop(['NoOfMonths_PartACov','NoOfMonths_PartBCov'],axis=1, inplace= True)

In [90]:
train_fin2['DOB']= pd.to_datetime(train_fin2['DOB'])
train_fin2['DOD']= pd.to_datetime(train_fin2['DOD'])

cv_fin2['DOB']= pd.to_datetime(cv_fin2['DOB'])
cv_fin2['DOD']= pd.to_datetime(cv_fin2['DOD'])

In [91]:
#Calculating the age of each of the patients
train_fin2['age']= np.zeros(len(train_fin2['DOB']))

a_max_tr= train_fin2['DOD'].max()

for i in range(len(train_fin2['DOB'])):
    if pd.isnull(train_fin2['DOD'][i])== True:
        train_fin2['age'][i]= (a_max_tr-train_fin2['DOB'][i])/timedelta(days=365)
    else:
        train_fin2['age'][i]= (train_fin2['DOD'][i]-train_fin2['DOB'][i])/timedelta(days=365)

In [92]:
cv_fin2['age']= np.zeros(len(cv_fin2['DOB']))

a_max_cv= cv_fin2['DOD'].max()

for i in range(len(cv_fin2['DOB'])):
    if pd.isnull(cv_fin2['DOD'][i])== True:
        cv_fin2['age'][i]= (a_max_cv-train_fin2['DOB'][i])/timedelta(days=365)
    else:
        cv_fin2['age'][i]= (cv_fin2['DOD'][i]-cv_fin2['DOB'][i])/timedelta(days=365)


In [93]:
#Dropping the DOB and DOD columns from the dataset
train_fin2.drop(['DOB','DOD'], axis=1,inplace=True)
cv_fin2.drop(['DOB','DOD'], axis=1,inplace=True)

In [94]:
#Imputing the ClaimAditDiagosis variable with 0
train_fin2['ClmAdmitDiagnosisCode']= train_fin2['ClmAdmitDiagnosisCode'].fillna(0)
cv_fin2['ClmAdmitDiagnosisCode']= cv_fin2['ClmAdmitDiagnosisCode'].fillna(0)

In [95]:
#Response Encoding of the ClaimsAdmitDiagnosis variable due to the presence of large number of categories in the variable
train_fin2['CADC_Yes']= np.zeros(len(train_fin2['ClmAdmitDiagnosisCode']))
train_fin2['CADC_No']= np.zeros(len(train_fin2['ClmAdmitDiagnosisCode']))
cv_fin2['CADC_Yes']= np.zeros(len(cv_fin2['ClmAdmitDiagnosisCode']))
cv_fin2['CADC_No']= np.zeros(len(cv_fin2['ClmAdmitDiagnosisCode']))

train_fin2['CADC_Yes'],train_fin2['CADC_No'],cv_fin2['CADC_Yes'],cv_fin2['CADC_No'],cadc_dict= response_coding(train_fin2,cv_fin2,'ClmAdmitDiagnosisCode','PotentialFraud')


In [96]:
#Dropping the ClmAdmitDiagnosisCode from the dataset
train_fin2.drop(['ClmAdmitDiagnosisCode'], axis=1, inplace=True)
cv_fin2.drop(['ClmAdmitDiagnosisCode'], axis=1, inplace=True)

In [97]:
#Replacing the Y and 0 in the RenalDiseaseIdicator varibale with 1 and 0
train_fin2['RenalDiseaseIndicator']= train_fin2['RenalDiseaseIndicator'].map({'Y':1,'0':0})
cv_fin2['RenalDiseaseIndicator']= cv_fin2['RenalDiseaseIndicator'].map({'Y':1,'0':0})

In [98]:
nam_cols= ['ChronicCond_Alzheimer','ChronicCond_Heartfailure','ChronicCond_KidneyDisease','ChronicCond_Cancer','ChronicCond_ObstrPulmonary','ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis','ChronicCond_rheumatoidarthritis','ChronicCond_stroke']

In [99]:
for i in nam_cols:
    train_fin2[i]= train_fin2[i].map({1:0,2:1})
    cv_fin2[i]= cv_fin2[i].map({1:0,2:1})


In [100]:
train_fin2['Gender']= train_fin2['Gender'].map({1:'G1',2:'G2'})
train_fin2['Race']= train_fin2['Race'].map({1:'R1',2:'R2',3:'R3',5:'R4'})

cv_fin2['Gender']= cv_fin2['Gender'].map({1:'G1',2:'G2'})
cv_fin2['Race']= cv_fin2['Race'].map({1:'R1',2:'R2',3:'R3',5:'R4'})


In [101]:
#Imputing the DeductibleAmtPaid with Median strategy using SimpleImpute
num_imp= SimpleImputer(missing_values= np.nan, strategy= 'median')
train_fin2["DeductibleAmtPaid"]= num_imp.fit_transform(train_fin2['DeductibleAmtPaid'].values.reshape(-1,1))[:,0]
cv_fin2["DeductibleAmtPaid"]= num_imp.transform(cv_fin2['DeductibleAmtPaid'].values.reshape(-1,1))[:,0]

In [102]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n3.pkl','wb') as tr_df:
    pickle.dump(train_fin2,tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n3.pkl','wb') as cv_df:
    pickle.dump(cv_fin2,cv_df)

In [110]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n3.pkl','rb') as tr_df:
    train_fin3= pickle.load(tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n3.pkl','rb') as cv_df:
    cv_fin3= pickle.load(cv_df)

In [111]:
num_cols=['InscClaimAmtReimbursed','DeductibleAmtPaid','IPAnnualReimbursementAmt','IPAnnualDeductibleAmt','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt','age']


In [112]:
train_fin3.head()

,InscClaimAmtReimbursed,DeductibleAmtPaid,Gender,Race,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,...,BID_No,CID_Yes,CID_No,Pvr_Yes,Pvr_No,Ap_Yes,Ap_No,age,CADC_Yes,CADC_No
0,90,0.0,G2,R1,0,1,1,0,1,1,...,0.875000,0.090909,0.909091,0.000540,0.999460,0.003215,0.996785,71.550685,0.361446,0.638554
1,5000,1068.0,G1,R1,1,0,1,0,1,1,...,0.090909,0.909091,0.090909,0.999011,0.000989,0.952381,0.047619,73.383562,0.571667,0.428333
2,400,0.0,G1,R1,0,1,0,1,1,1,...,0.500000,0.909091,0.090909,0.999965,0.000035,0.941176,0.058824,79.054795,0.365412,0.634588
3,30,0.0,G2,R2,0,1,1,0,1,1,...,0.012346,0.909091,0.090909,0.999947,0.000053,0.997375,0.002625,53.284932,0.365412,0.634588
4,2000,1068.0,G2,R1,0,0,0,1,1,1,...,0.800000,0.909091,0.090909,0.999174,0.000826,0.998959,0.001041,82.224658,0.520833,0.479167


In [113]:
cat_train= train_fin3.select_dtypes(include=['object']).copy()
cat_train.drop(['PotentialFraud'],axis=1,inplace=True)
cat_train.head()

,Gender,Race,IllnessNature
0,G2,R1,operating
1,G1,R1,simple
2,G1,R1,simple
3,G2,R2,operating
4,G2,R1,comorbid


In [109]:
train_fin3.head()

,InscClaimAmtReimbursed,DeductibleAmtPaid,Gender,Race,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,...,CID_Yes,CID_No,Pvr_Yes,Pvr_No,Ap_Yes,Ap_No,age,CADC_Yes,CADC_No,age_bins
0,90,0.0,G2,R1,0,1,1,0,1,1,...,0.090909,0.909091,0.000540,0.999460,0.003215,0.996785,71.550685,0.361446,0.638554,60+
1,5000,1068.0,G1,R1,1,0,1,0,1,1,...,0.909091,0.090909,0.999011,0.000989,0.952381,0.047619,73.383562,0.571667,0.428333,60+
2,400,0.0,G1,R1,0,1,0,1,1,1,...,0.909091,0.090909,0.999965,0.000035,0.941176,0.058824,79.054795,0.365412,0.634588,60+
3,30,0.0,G2,R2,0,1,1,0,1,1,...,0.909091,0.090909,0.999947,0.000053,0.997375,0.002625,53.284932,0.365412,0.634588,50s
4,2000,1068.0,G2,R1,0,0,0,1,1,1,...,0.909091,0.090909,0.999174,0.000826,0.998959,0.001041,82.224658,0.520833,0.479167,80+


In [114]:
train_fin3.drop(['PotentialFraud'],axis=1, inplace=True)
cv_fin3.drop(['PotentialFraud'],axis=1,inplace=True)

In [115]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n4.pkl','wb') as tr_df:
    pickle.dump(train_fin3,tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n4.pkl','wb') as cv_df:
    pickle.dump(cv_fin3,cv_df)

In [116]:
with open('/content/drive/MyDrive/Colab Notebooks/train_fin_n4.pkl','rb') as tr_df:
    train_fin4= pickle.load(tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_fin_n4.pkl','rb') as cv_df:
    cv_fin4= pickle.load(cv_df)

In [117]:
train_fin4.head()

,InscClaimAmtReimbursed,DeductibleAmtPaid,Gender,Race,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,...,BID_No,CID_Yes,CID_No,Pvr_Yes,Pvr_No,Ap_Yes,Ap_No,age,CADC_Yes,CADC_No
0,90,0.0,G2,R1,0,1,1,0,1,1,...,0.875000,0.090909,0.909091,0.000540,0.999460,0.003215,0.996785,71.550685,0.361446,0.638554
1,5000,1068.0,G1,R1,1,0,1,0,1,1,...,0.090909,0.909091,0.090909,0.999011,0.000989,0.952381,0.047619,73.383562,0.571667,0.428333
2,400,0.0,G1,R1,0,1,0,1,1,1,...,0.500000,0.909091,0.090909,0.999965,0.000035,0.941176,0.058824,79.054795,0.365412,0.634588
3,30,0.0,G2,R2,0,1,1,0,1,1,...,0.012346,0.909091,0.090909,0.999947,0.000053,0.997375,0.002625,53.284932,0.365412,0.634588
4,2000,1068.0,G2,R1,0,0,0,1,1,1,...,0.800000,0.909091,0.090909,0.999174,0.000826,0.998959,0.001041,82.224658,0.520833,0.479167


In [118]:
#Creat Dummies for the Gender, Race and the IllnessNature Variables in the Train Dataset
gender_dummies= pd.get_dummies(train_fin4['Gender'])
train_fin4= train_fin4.join(gender_dummies)

In [119]:
race_dummies= pd.get_dummies(train_fin4['Race'])
train_fin4= train_fin4.join(race_dummies)

In [120]:
illn_dummies= pd.get_dummies(train_fin4['IllnessNature'])
train_fin4= train_fin4.join(illn_dummies)

In [121]:
train_fin4.drop(['Gender','Race','IllnessNature'], axis=1, inplace=True)

In [122]:
train_fin4.head()

,InscClaimAmtReimbursed,DeductibleAmtPaid,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,...,G1,G2,R1,R2,R3,R4,comorbid,operating,operating&comorbid,simple
0,90,0.0,0,1,1,0,1,1,0,0,...,0,1,1,0,0,0,0,1,0,0
1,5000,1068.0,1,0,1,0,1,1,1,0,...,1,0,1,0,0,0,0,0,0,1
2,400,0.0,0,1,0,1,1,1,1,0,...,1,0,1,0,0,0,0,0,0,1
3,30,0.0,0,1,1,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,0
4,2000,1068.0,0,0,0,1,1,1,1,1,...,0,1,1,0,0,0,1,0,0,0


In [123]:
#Creating Dummies for the Gender, Race and the IllnessNature Variables in the CV Dataset
cv_gender_dummies= pd.get_dummies(cv_fin4['Gender'])
cv_fin4= cv_fin4.join(cv_gender_dummies)


In [124]:
cv_race_dummies= pd.get_dummies(cv_fin4['Race'])
cv_fin4= cv_fin4.join(cv_race_dummies)

In [125]:
cv_illn_dummies= pd.get_dummies(cv_fin4['IllnessNature'])
cv_fin4= cv_fin4.join(cv_illn_dummies)

In [126]:
cv_fin4.drop(['Gender','Race','IllnessNature'], axis=1, inplace=True)

In [127]:
cv_fin4.head()

,InscClaimAmtReimbursed,DeductibleAmtPaid,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,...,G1,G2,R1,R2,R3,R4,comorbid,operating,operating&comorbid,simple
0,300,0.0,0,1,1,1,1,1,1,0,...,1,0,1,0,0,0,0,0,1,0
1,80,50.0,0,1,1,1,1,1,1,1,...,0,1,0,1,0,0,0,0,0,1
2,200,0.0,0,1,1,1,1,1,1,1,...,0,1,1,0,0,0,0,0,0,1
3,200,0.0,0,1,0,1,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
4,300,0.0,0,0,1,1,1,1,1,0,...,1,0,1,0,0,0,0,0,0,1


In [128]:
#Scaling the Numerical Features using the SKLearn StandardScaler
scaler= StandardScaler()
num_cols=['InscClaimAmtReimbursed','DeductibleAmtPaid','IPAnnualReimbursementAmt','IPAnnualDeductibleAmt','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt','#_Procedures','#_DiagnosisCodes','HospitalWeeks','ClaimWeeks','age']

for i in tqdm(num_cols):
    scaler.fit(train_fin4[i].values.reshape(-1,1))
    train_fin4[i]= scaler.transform(train_fin4[i].values.reshape(-1,1))
    cv_fin4[i]= scaler.transform(cv_fin4[i].values.reshape(-1,1)) 

100%|██████████| 11/11 [00:00<00:00, 39.46it/s]


In [129]:
print(train_fin4.shape)
print(train_y.shape)
print(cv_fin4.shape)
print(cv_y.shape)

(446568, 49)
(446568,)
(111643, 49)
(111643,)


In [130]:
#Replacing the Yes and No with 1 and 0 in the Train_y and CV_y datasets
train_y= train_y.map({'Yes':1,'No':0})
cv_y= cv_y.map({'Yes':1,'No':0})
print(train_y.head())
print('='*50)
print(cv_y.head())

0    0
1    1
2    1
3    1
4    1
Name: PotentialFraud, dtype: int64
0    1
1    0
2    0
3    0
4    1
Name: PotentialFraud, dtype: int64


In [131]:
with open('/content/drive/MyDrive/Colab Notebooks/train_hc_fin.pkl','wb') as tr_df:
    pickle.dump(train_fin4,tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_hc_fin.pkl','wb') as cv_df:
    pickle.dump(cv_fin4,cv_df)

In [132]:
with open('/content/drive/MyDrive/Colab Notebooks/train_hc_fin.pkl','rb') as tr_df:
    train_fin4= pd.read_pickle(tr_df)
with open('/content/drive/MyDrive/Colab Notebooks/cv_hc_fin.pkl','rb') as cv_df:
    cv_fin4= pd.read_pickle(cv_df)

In [133]:

with open('/content/drive/MyDrive/Colab Notebooks/train_y.pkl','rb') as tr_y:
    train_y= pd.read_pickle(tr_y)
with open('/content/drive/MyDrive/Colab Notebooks/cv_y.pkl','rb') as cv_y:
    cv_y= pd.read_pickle(cv_y)

In [135]:
print(train_fin4.shape)
print(train_y.shape)
print(cv_fin4.shape)
print(cv_y.shape)

(446568, 49)
(446568,)
(111643, 49)
(111643,)
